In [1]:
import os
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
import librosa
from matplotlib import cm
from librosa import display
import pylab
from scipy.io import wavfile

In [2]:
audio_genres = pd.read_csv("audiofile_genres.csv", sep=";")
audio_genres.head()

,Audio_Filenames,track_genre_1,track_genre_2
0,4em001_01_Mideast-Darkness_703611.wav,Atmosphere,World Music
1,4em001_02_Desert-Search_703612.wav,Atmosphere,World Music
2,4em001_03_Qasidah_703613.wav,Atmosphere,World Music
3,4em001_04_Desert-War_703614.wav,Atmosphere,World Music
4,4em001_05_Desert-War---Lite_703615.wav,Atmosphere,World Music


In [3]:
occur_col1 = audio_genres["track_genre_1"].value_counts()
occur_col1

Atmosphere            8992
Filmscore             8305
Rock                  4405
Electronica           3356
Hip Hop, Rap          2364
Country, Folk         2305
Pop                   2042
Classical Music       1724
Easy Listening        1620
World Music           1573
Ambient, Chill        1503
Dance                 1394
Indie, Alternative    1153
Funk, Soul            1069
Jazz                  1005
House                  761
Latin                  524
Techno                 375
Big Band               371
Blues                  336
60ies                  333
Drum'n'Bass            327
Military               261
R'n'B                  200
German Folk            189
80ies                  181
Historical Music       169
Swing                  127
Sound Design           126
Christmas              104
Acoustic               100
Orchestral              95
Hard, Heavy             81
Kids                    78
70ies                   75
Religious Music         73
Reggae, Ska             63
T

In [4]:
occur_col2 = audio_genres["track_genre_2"].value_counts()
occur_col2

Pop                   4437
Filmscore             4424
Electronica           3624
Ambient, Chill        3100
Classical Music       1381
World Music           1341
Indie, Alternative     688
Orchestral             655
Kids                   628
Trailer                612
Acoustic               600
Hard, Heavy            555
Sound Design           520
Jazz                   503
Christmas              409
Dance                  386
Drone                  369
Blues                  352
Swing                  331
Easy Listening         311
Piano                  296
Atmosphere             293
R'n'B                  292
70ies                  272
Funk, Soul             228
Song                   223
Dubstep                221
Rock'n'Roll            206
80ies                  189
Religious Music        182
Pizzicato              180
Percussion             155
Punk                   147
60ies                  144
Reggae, Ska            134
House                  128
90ies                  122
H

In [5]:
occur_tot = occur_col1 + occur_col2
occur_tot.sort_values(ascending=False)

Filmscore             12729.0
Atmosphere             9285.0
Electronica            6980.0
Pop                    6479.0
Ambient, Chill         4603.0
Rock                   4469.0
Classical Music        3105.0
World Music            2914.0
Hip Hop, Rap           2433.0
Country, Folk          2413.0
Easy Listening         1931.0
Indie, Alternative     1841.0
Dance                  1780.0
Jazz                   1508.0
Funk, Soul             1297.0
House                   889.0
Orchestral              750.0
Kids                    706.0
Acoustic                700.0
Blues                   688.0
Trailer                 657.0
Sound Design            646.0
Hard, Heavy             636.0
Latin                   572.0
Christmas               513.0
R'n'B                   492.0
60ies                   477.0
Swing                   458.0
Drone                   412.0
Big Band                403.0
Techno                  388.0
80ies                   370.0
Drum'n'Bass             352.0
70ies     

In [6]:
audio_genres["track_genre_1"].replace({"Big Band": "Others", "German Folk": "Others", "Techno": "Others", "80ies": "Others", "Drum'n'Bass": "Others", "70ies": "Others", "Piano": "Others", "Historical Music": "Others", "Military": "Others", "Dubstep": "Others", "Religious Music": "Others", "Rock'n'Roll": "Others", "Song": "Others", "Pizzicato": "Others", "Percussion": "Others", "German, Folk": "Others", "Reggae, Ska": "Others", "Punk": "Others", "90ies": "Others", "Disco": "Others", "Industrial": "Others", "20ies": "Others", "Breakbeat": "Others", "A Cappella": "Others", "50ies": "Others", "40ies": "Others", "30ies": "Others"}, inplace=True)
newoccur_col1 = audio_genres["track_genre_1"].value_counts()
newoccur_col1

Atmosphere            8992
Filmscore             8305
Rock                  4405
Electronica           3356
Others                2456
Hip Hop, Rap          2364
Country, Folk         2305
Pop                   2042
Classical Music       1724
Easy Listening        1620
World Music           1573
Ambient, Chill        1503
Dance                 1394
Indie, Alternative    1153
Funk, Soul            1069
Jazz                  1005
House                  761
Latin                  524
Blues                  336
60ies                  333
R'n'B                  200
Swing                  127
Sound Design           126
Christmas              104
Acoustic               100
Orchestral              95
Hard, Heavy             81
Kids                    78
Trailer                 45
Drone                   43
Name: track_genre_1, dtype: int64

In [7]:
audio_genres["track_genre_2"].replace({"Big Band": "Others", "German Folk": "Others", "Techno": "Others", "80ies": "Others", "Drum'n'Bass": "Others", "70ies": "Others", "Piano": "Others", "Historical Music": "Others", "Military": "Others", "Dubstep": "Others", "Religious Music": "Others", "Rock'n'Roll": "Others", "Song": "Others", "Pizzicato": "Others", "Percussion": "Others", "German, Folk": "Others", "Reggae, Ska": "Others", "Punk": "Others", "90ies": "Others", "Disco": "Others", "Industrial": "Others", "20ies": "Others", "Breakbeat": "Others", "A Cappella": "Others", "50ies": "Others", "40ies": "Others", "30ies": "Others"}, inplace=True)
newoccur_col2 = audio_genres["track_genre_2"].value_counts()
newoccur_col2

Pop                   4437
Filmscore             4424
Electronica           3624
Ambient, Chill        3100
Others                2951
Classical Music       1381
World Music           1341
Indie, Alternative     688
Orchestral             655
Kids                   628
Trailer                612
Acoustic               600
Hard, Heavy            555
Sound Design           520
Jazz                   503
Christmas              409
Dance                  386
Drone                  369
Blues                  352
Swing                  331
Easy Listening         311
Atmosphere             293
R'n'B                  292
Funk, Soul             228
60ies                  144
House                  128
Country, Folk          108
Hip Hop, Rap            69
Rock                    64
Latin                   48
Name: track_genre_2, dtype: int64

In [8]:
newoccur_tot = newoccur_col1 + newoccur_col2
newoccur_tot.sort_values(ascending=False)

Filmscore             12729
Atmosphere             9285
Electronica            6980
Pop                    6479
Others                 5407
Ambient, Chill         4603
Rock                   4469
Classical Music        3105
World Music            2914
Hip Hop, Rap           2433
Country, Folk          2413
Easy Listening         1931
Indie, Alternative     1841
Dance                  1780
Jazz                   1508
Funk, Soul             1297
House                   889
Orchestral              750
Kids                    706
Acoustic                700
Blues                   688
Trailer                 657
Sound Design            646
Hard, Heavy             636
Latin                   572
Christmas               513
R'n'B                   492
60ies                   477
Swing                   458
Drone                   412
dtype: int64

In [9]:
audio_genres.set_index('Audio_Filenames', inplace=True)

In [ ]:
for f in audio_genres.index:
    rate, signal = wavfile.read('wav/'+f)
    audio_genres.at[f, 'length'] = signal.shape[0]/rate

In [ ]:
audio_genres.sort_values('length',ascending=True)

In [ ]:
audio_genres = audio_genres.drop(audio_genres[audio_genres.length < 40].index)

In [ ]:
audio_genres = audio_genres.reset_index('Audio_Filenames')

In [ ]:
audio_genres.sort_values('length',ascending=True)

In [ ]:
two_columns = [audio_genres["track_genre_1"], audio_genres["track_genre_2"]]
headers = ["genre_1", "genre_2"]
two_columns = pd.concat(two_columns, axis=1, keys=headers)
two_columns

In [ ]:
two_columns['genre_2'].fillna('', inplace=True)
two_columns

In [ ]:
two_columns.values.sort()
two_columns

In [ ]:
trackgenre1 = two_columns["genre_1"]
trackgenre2 = two_columns["genre_2"]
audio_genres['multi_genres'] = trackgenre1 + ' ' + trackgenre2

In [ ]:
pd.set_option('display.max_rows', 450)
audio_genres['multi_genres'].value_counts()

In [ ]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == ' Filmscore'].index, size=2750, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

In [ ]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == 'Atmosphere Filmscore'].index, size=2050, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

In [ ]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == ' Atmosphere'].index, size=1600, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

In [ ]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == ' Electronica'].index, size=1000, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

In [ ]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == ' Rock'].index, size=500, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

In [ ]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == 'Atmosphere Electronica'].index, size=460, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

In [ ]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == ' Pop'].index, size=440, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

In [ ]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == ' Classical Music'].index, size=20, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

In [ ]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == 'Ambient, Chill Filmscore'].index, size=420, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

In [ ]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == ' Others'].index, size=380, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

In [ ]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == ' World Music'].index, size=380, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

In [ ]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == ' Hip Hop, Rap'].index, size=380, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

In [ ]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == 'Electronica Filmscore'].index, size=360, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

In [ ]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == 'Filmscore Others'].index, size=380, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

In [ ]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == 'Ambient, Chill Atmosphere'].index, size=250, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

In [ ]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == 'Pop Rock'].index, size=240, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

In [ ]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == ' Ambient, Chill'].index, size=220, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

In [ ]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == ' Country, Folk'].index, size=150, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

In [ ]:
np.random.seed(10)
drop_indices = np.random.choice(audio_genres[audio_genres['multi_genres'] == 'Filmscore Pop'].index, size=250, replace=False)
audio_genres.drop(labels=drop_indices, axis=0, inplace=True)
audio_genres.reset_index(drop=True, inplace=False)

In [ ]:
audio_genres.to_csv('clean_audio_genres.csv', sep=';')